In [17]:
#必要なライブラリのインポート
import pandas as pd
import numpy as np
import PIL.Image
import PIL.ImageTk
from tkinter import filedialog
import tkinter as tk
from PIL import ImageFont
from PIL import  ImageDraw

In [18]:
global file_name

In [19]:
#テキストデータを出力する関数
#テキストデータを出力する関数
def MaketextData(MosaicData,file_name):
    #同じ名前のファイルが存在していた場合，末尾に数字をつけて保存する
    num = 0
    while True:
        try:
            if num == 0:
                adress = "TextData/" + file_name  + ".txt"
                f=open(adress,"x")
                #print("ファイル名:",file_name)
                break
            else :
                adress = "TextData/" + file_name  + str(num) + ".txt"
                f=open(adress,"x")
                #print("ファイル名:",file_name)
            break
        except:
            num += 1
    f.write(MosaicData)
    f.close

In [20]:
#モザイクアートデータを出力する関数
def ShowMosaicArt(MosaicData,file_name):
    #結果の文字列を作成
    result = ""
    for i in MosaicData:
        for j in i:
            result = result + str(j)
        result = result + "\n"

    #文字列を.txtに出力
    MaketextData(result,file_name)

    """ #結果を表示する用のtkinterウィンドウを用意
    root_result = tk.Toplevel()
    root_result.geometry(("1000x1000"))
    root_result.title("出力結果")
    #結果を出力するラベル
    image_lbl = tk.Label(root_result)

    #画像を準備
    img = PIL.Image.new("L",(1000,1000),color=255)
    draw = PIL.ImageDraw.Draw(img)

    #フォント情報を作成
    #フォント情報を作成
    #等倍(80,240)なら8.2
    font_size = 8.2
    font = PIL.ImageFont.truetype("MSMINCHO.TTF",font_size)

    #フォントを書く
    draw.text((0,0),result,font=font)

    #tkinterで表示できるように処理
    img = PIL.ImageTk.PhotoImage(img.resize((1000,1000),resample=0))

    #画像を表示
    image_lbl.configure(image=img)
    image_lbl.image = img

    image_lbl.pack()

    root_result.mainloop() """

In [21]:
#画像データの数値リストからモザイクアートを作成
def CreateMosaicArt(image_data,file_name,image_width,image_height):
    #データセット(csv)を取得
    df = pd.read_csv("DataSets_MSGothic.csv",header=None)
    df = df.sort_values(by=3,ascending=False)
    
    #pandasのデータフレームからリストを作成
    index_list = list(df[0])
    lett_list = list(df[1])
    num_list = list(df[4])
    #print(index_list)
    #print(lett_list)
    #print(num_list)

    #出力するデータ(正解データ)
    #0埋めで100×100のデータを作成
    result = np.zeros((image_height,image_width),dtype=float) #数値を格納するデータ
    result_lett = np.zeros((image_height,image_width),dtype=str) #文字列を格納するデータ

    for i ,first_dim in enumerate(result):
        for j , second_dim in enumerate(first_dim):
            #画像データを一つずつ取り出す
            data = image_data[i][j]

            #取り出したデータに最も近いデータを探す
            dis = 10 #無限大として定義
            result_temp = 0 #数値データ

            for k in num_list:
                #差の絶対値を計算
                dis_temp = abs(data-k)

                #暫定の最近値よりも近いものが見つかったら中身書き換え
                if dis_temp < dis :
                    dis = dis_temp #最近距離
                    result_temp = k #格納する数値

            result[i][j] = result_temp
            result_lett[i][j] = lett_list[num_list.index(result_temp)]

    #print(result)
    #print(result_lett)

    #結果を表示する
    ShowMosaicArt(result_lett,file_name)

In [22]:
#画像を数値リストに変換する
def ImageToNum(path,file_name):
    #画像をグレースケールに変換する
    gray_image = PIL.Image.open(path).convert("L")
    width , height = gray_image.size
    print(f"幅:{width},高さ:{height}")
    
    #デフォルト240,120

    image_width = 360
    image_height = 180
    
    #画像のサイズを分析し，サイズを決める
    if width >= height : #横長または正方形
        image_width = image_width
        image_height = int(image_width * height / width /2)
        
    else: #縦長
        image_height = image_height 
        image_width = int(image_height * width / height) * 2
        
    print(f"画像幅:{image_width},画像高さ:{image_height}")
    
    #240×130のデータに変換
    gray_image = gray_image.resize((image_width,image_height),PIL.Image.Resampling.LANCZOS)

    #画像をtkinter上に表示する
    #PILのデータをtkinterで表示できるように変換
    image_data = PIL.ImageTk.PhotoImage(gray_image.resize((1000,1000),resample=0))
    image_lbl.configure(image=image_data)
    image_lbl.image = image_data

    #画像データを数値リストに変換する
    num_data = np.asarray(gray_image,dtype=int)
    #データを0~1で正規化
    num_data = 255 -num_data
    num_data = num_data / 255
    #print(num_data)
    #print(num_data.shape)

    #print("ファイル名:",file_name)

    #次の関数にわたす
    CreateMosaicArt(num_data,file_name,image_width,image_height)

In [23]:
#ファイルダイアログを作成する関数
def OpenFile():
    file_path = filedialog.askopenfilename()
    file_name = ""
    if file_path :
        file_name = file_path.split("/")[-1].split(".")[0]
        ImageToNum(file_path,file_name)
        #print(file_name)
        

In [24]:
#tkinterの画面を作成
root = tk.Tk()
root.geometry(("1000x1000"))
root.title("画像変換結果")

#ファイルダイアログを開くボタン
open_file_btn = tk.Button(text="画像を選択してください",command=OpenFile)
#画像を表示するラベル
image_lbl = tk.Label()

#配置
open_file_btn.pack()
image_lbl.pack()

root.mainloop()

幅:727,高さ:727
画像幅:360,画像高さ:180
